This notebook generates a summarization using the sentences extracted from the previous steps. The BART model is used to generate text conditioned on the given input. See [documentations](https://huggingface.co/transformers/model_doc/bart.html#transformers.BartForConditionalGeneration) and [paper](https://arxiv.org/pdf/1910.13461) for model descriptions.

In [1]:
from functools import partial

import pandas as pd
import torch
import transformers

from IPython.display import Markdown, display

In [2]:
# tested with package versions: pandas==0.25.1   pytorch==1.4.0   transformers==2.7.0
print(pd.__version__)
print(torch.__version__)
print(transformers.__version__)

0.25.1
1.4.0
2.7.0


In [3]:
def printmd(string):
    display(Markdown(string))

### Load data from the previous step

In [4]:
qas = pd.read_csv('Task4_question1.csv', usecols=[1,2,3])
category_dfn = pd.read_csv('category question.csv', 
                           header=None, 
                           index_col=0,
                           names=['category', 'description']).to_dict()
qas['question'] = qas['category'].map(category_dfn['description'])

In [5]:
qas

,title,sent,category,question
0,HIV protease inhibitor nelfinavir inhibits rep...,Quantitative RT-PCR analysis showed that nelfi...,antihiv,Does anti HIV drug can cure COVID-2019
1,HIV protease inhibitor nelfinavir inhibits rep...,Our results suggest that nelfinavir should be ...,antihiv,Does anti HIV drug can cure COVID-2019
2,HIV protease inhibitor nelfinavir inhibits rep...,Expression of viral antigens was much lower in...,antihiv,Does anti HIV drug can cure COVID-2019
3,HIV protease inhibitor nelfinavir inhibits rep...,Here we report that the HIV-1 protease inhibit...,antihiv,Does anti HIV drug can cure COVID-2019
4,Molecular Modeling Evaluation of the Binding E...,"Three anti-HIV drugs, ritonavir, lopinavir and...",antihiv,Does anti HIV drug can cure COVID-2019
...,...,...,...,...
1479,Activation of glial cells by human coronavirus...,This virus also modulated the activity of matr...,interleukin,Does interleukin involved in COVID-2019
1480,TGEV nucleocapsid protein induces cell cycle a...,Further studies showed that p53 inhibitor atte...,interleukin,Does interleukin involved in COVID-2019
1481,Nuclear targeting of the betanodavirus B1 prot...,B1 nuclear localization correlated with upregu...,interleukin,Does interleukin involved in COVID-2019
1482,The involvement of survival signaling pathways...,Therefore we investigated the role of phosphoi...,interleukin,Does interleukin involved in COVID-2019


In [6]:
qas = qas.groupby('question').agg({'title': list, 'sent': list})

In [7]:
qas

,title,sent
question,,
Does anti HCV drug can cure COVID-2019,[Prediction of the SARS-CoV-2 (2019-nCoV) 3C-l...,[The drugs Epclusa (velpatasvir/sofosbuvir) an...
Does anti HIV drug can cure COVID-2019,[HIV protease inhibitor nelfinavir inhibits re...,[Quantitative RT-PCR analysis showed that nelf...
Does anti-parasite drug can cure COVID-2019,[Hydroxychloroquine and azithromycin as a trea...,[Azithromycin added to hydroxychloroquine was ...
"Does antibiotic drug can cure COVID-2019,",[Hydroxychloroquine and azithromycin as a trea...,"[Depending on their clinical presentation, azi..."
Does cancer drug can cure COVID-2019,[MERS-CoV pathogenesis and antiviral efficacy ...,"[Unlike in established cells, toremifene had m..."
Does interleukin involved in COVID-2019,[Transmissible Gastroenteritis Virus Infection...,"[In vivo, TGEV primarily targets and infects i..."
"Does other anti virus drug can cure COVID-2019,",[Synergistic antiviral effect of Galanthus niv...,"[However, when the two agents were added toget..."
Does other immune supprevisive drug can cure COVID-2019,[A Systematic Review of therapeutic agents for...,[Only one such study is planned for randomizat...


### load pretrained BART model for summarization

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
bart_summarizer = transformers.BartForConditionalGeneration.from_pretrained('bart-large-cnn').to(device)
summarizer_tokenizer = transformers.BartTokenizer.from_pretrained('bart-large-cnn')

In [10]:
def summarize(sentences, summarizer, tokenizer):
    device = summarizer.device
    tokens = tokenizer.batch_encode_plus([' '.join(sentences)], return_tensors='pt', max_length=1024)
    summary_tokens = summarizer.generate(input_ids=tokens['input_ids'].to(device), 
                                         attention_mask=tokens['attention_mask'].to(device),
                                         pad_token_id=tokenizer.pad_token_id).cpu()
    summary_text = tokenizer.decode(summary_tokens.squeeze(), skip_special_tokens=True)
    return summary_text.strip()

In [11]:
summarize_ = partial(summarize, summarizer=bart_summarizer, tokenizer=summarizer_tokenizer)
qas['summarization'] = qas['sent'].apply(summarize_)

In [23]:
for q, row in qas.iterrows():
    printmd(f'**Question: {q.strip()}**')
    printmd(f'*Summarization:{row["summarization"]}*')
#     print('Raw input:', row['sent'])
    print('\n\n')

**Question: Does anti HCV drug can cure COVID-2019**

*Summarization:Epclusa and Harvoni could be very effective owing to their dual inhibitory actions on two viral enzymes. Among these, the antivirals ledipasvir or velpatasvir are particularly attractive as therapeutics to combat the new coronavirus with minimal side effects, commonly fatigue and headache.*

**Question: Does anti HIV drug can cure COVID-2019**

*Summarization:HIV-1 protease inhibitor, nelfinavir, strongly inhibited replication of the SARS coronavirus (SARS-CoV) Expression of viral antigens was much lower in infected cells treated with nELFinavir than in untreated infected cells. Our results suggest that nelf Scandinavir should be examined clinically for the treatment of SARS and has potential as a good lead compound for designing anti-SARS drugs.*

**Question: Does anti-parasite drug can cure COVID-2019**

...

*Summarization:P2X7 antagonist A438079 suppressed NLRP3/ASC/caspase 1 activation, production of IL-1β, IL-17A and IFN-γ and neutrophil infiltration but not IL-10. ORF5 protein triggers the phosphorylation of PERK, eIF2α and the expression of downstream transcription factor ATF4.*

**Question: Does other anti virus drug can cure COVID-2019,**

*Summarization:There are no effective antiviral drugs for many viral infections. Baicalein inhibited in vitro replication of influenza A viruses pandemic 2009 H1N1 (IC(50) = 0.018 μM) and seasonal 2007 H1n1 using plaque reduction assays. Of the 42 lipophilic teicoplanin pseudoaglycon derivatives tested, about half showed broad activity against influenza A and B viruses, with some having reasonable or no cytotoxicity.*

**Question: Does other immune supprevisive drug can cure COVID-2019**

*Summarization:A unique set of 207 genes was dysregulated early and permanently throughout infection with HCoV-EMC. These genes were used to predict potential antiviral compounds, including kinase inhibitors and glucocorticoids. Using network proximity analyses, we identified 135 putative repurposable drugs for the potential prevention and treatment of HCoVs. Two SARS-CoV PLpro inhibitors are able to inhibit MERS-Cov PLpro genes and pathways.*